# Final Project: Creating a Spam Detection Program

For my final project I will be creating a spam detection program. I will be using a dataset of around 5,500 sample text messages that have been labeled "ham" (not spam) or "spam". My job is to create a machine learning algorithm that can most accurately predict whether or not a given text is spam.

### Importing the required modules/packages

In [115]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import re
import nltk
from nltk.corpus import stopwords
import string
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

### Loading file and looking into the dimensions of data

In [2]:
raw_data = pd.read_csv("SMSSpamCollection.tsv",sep='\t',names=['label','text'])
pd.set_option('display.max_colwidth',150)
raw_data.head(30)

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 0845281007...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your frie...
8,spam,WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 ...
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 080...


In [3]:
print(raw_data.shape)
pd.crosstab(raw_data['label'],columns = 'label',normalize=True)

(5572, 2)


col_0,label
label,
ham,0.865937
spam,0.134063


We can see here that we have an unbalanced data set. This means that it may be difficult to get an accuracy that is much better than the baseline, namely 86.59%, so we will have to rely on other metrics, like precision and recall. The general consensus is that it is better to let some spam get through the filter rather than accidentally filter out a legitimate email. This means I want to eliminate Type I errors, and so my goal is to keep precision as close to 100% as possible while also making recall as high as I can.

To do this I will make an fbeta scorer and set my beta equal to 0.1, which means that I am weighting precision 10 times as heavily as recall. While there are many values for beta possible, I chose one that would weight precision heavily enough to force it to stay around 100% without making it too heavy as to discount recall entirely.

This is the metric I will be using for all of my testing.

In [164]:
score=make_scorer(fbeta_score, beta=0.1)

## Step 1: Exploring the Data

I'm going to create an initial count vectorizer to get some insight on how many unique words we have as well as what some of them look like.

In [119]:
v=CountVectorizer()
v

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [120]:
f=v.fit_transform(raw_data.text)

In [121]:
f.shape

(5572, 8713)

We see that there are 8,713 unique words that makes up this text. Let's take a look at some of these words to get a sense of what we're dealing with.

In [122]:
print((v.get_feature_names()[:50]))

['00', '000', '000pes', '008704050406', '0089', '0121', '01223585236', '01223585334', '0125698789', '02', '0207', '02072069400', '02073162414', '02085076972', '021', '03', '04', '0430', '05', '050703', '0578', '06', '07', '07008009200', '07046744435', '07090201529', '07090298926', '07099833605', '07123456789', '0721072', '07732584351', '07734396839', '07742676969', '07753741225', '0776xxxxxxx', '07781482378', '07786200117', '077xxx', '078', '07801543489', '07808', '07808247860', '07808726822', '07815296484', '07821230901', '078498', '07880867867', '0789xxxxxxx', '07946746291', '0796xxxxxx']


## Step 2: Build and Optimize a Count Vectorizer for each Model

Now that the data is ready to vectorize, I will use a Grid Search to find the best vectorization for each model I am going to test. I've decided to limit my testing to 3 models: Random Forest, Multinomial Naive Bayes, and SVM.

The first thing I'm going to do is do a train-test split on my data that I can use to build my model. Then I will build a stratified k-fold to use in the cross-validation.

In [130]:
# I'm encoding the label so that I can use binary scoring to evaluate my model.

X=raw_data.text
y=raw_data.label.map({'ham':0,'spam':1})

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create stratified k-fold

kfold = StratifiedKFold(n_splits=5)

Before I use the numeric column of my data, I need to tune my vectorizer to each model. I'm going to do a Grid search cross-validation to find the best score for precision. Cross-validation is a good idea for text data because it is completely random so it shouldn't cause overfitting.

### Random Forest

In [165]:
p = Pipeline([('v', CountVectorizer()), ('forest', RandomForestClassifier())])

g = GridSearchCV(p, param_grid={'v__min_df':[1,2], 'v__ngram_range':  [(1,1), (1,2)], 'v__stop_words': [None, 'english'], 
                                'v__lowercase': [True, False], 'v__max_features':[None, 10000, 25000]}, cv=kfold, scoring=score)
g.fit(X, y)
g.best_estimator_

Pipeline(memory=None,
     steps=[('v', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        st...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [166]:
g.best_score_

0.9963127386150937

Based on this result, I was able to get my score to 99.6% using cross-validation, with the ideal parameters for the Count Vectorizer being 1-2 n-grams ,no stop words, and not converting letters to lowercase. Now I'm going to fit an ideal vectorizer to a Random Forest and to a simple train test split to see the confusion matrix so I get an idea of how many of each type of error I am getting.

In [253]:
v=CountVectorizer(lowercase=False, ngram_range=(1,2))
f=v.fit_transform(raw_data.text)
f.shape

(5572, 55728)

In [202]:
X_train_f=v.fit_transform(X_train)
X_test_f=v.transform(X_test)
forest=RandomForestClassifier()
forest.fit(X_train_f, y_train)
y_pred = forest.predict(X_test_f)
print('F-beta score: %s' %(fbeta_score(y_test, y_pred, beta=0.2)))

F-beta score: 0.9855728429985855


In [203]:
metrics.confusion_matrix(y_test, y_pred)

array([[1208,    0],
       [  51,  134]], dtype=int64)

Even without tuning my Random Forest I have been able to keep precision at 100%, but the recall isn't great. Hopefully tuning my model will improve the recall.

### Naive Bayes

In [254]:
p = Pipeline([('v', CountVectorizer()), ('nb', MultinomialNB())])

g = GridSearchCV(p, param_grid={'v__min_df':[1,3,5], 'v__lowercase':[True, False], 'v__ngram_range':  [(1,1), (1,2)], 'v__stop_words': [None, 'english'], 
                                'v__max_features':[None, 10000, 25000]}, cv=kfold, scoring=score)
g.fit(X, y)
g.best_estimator_

Pipeline(memory=None,
     steps=[('v', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [255]:
g.best_score_

0.9918029157032761

It appears that the ideal vectorizer for Naive Bayes is almost the same as the one for Random Forest Classifier, except for converting letters to lowercase.

In [224]:
v2=CountVectorizer(lowercase=True, ngram_range=(1,2))

In [225]:
X_train_f2=v2.fit_transform(X_train)
X_test_f2=v2.transform(X_test)
nb=MultinomialNB()
nb.fit(X_train_f2, y_train)
y_pred = nb.predict(X_test_f2)
print('F-beta score: %s' %(fbeta_score(y_test, y_pred, beta=0.1)))

F-beta score: 0.9826111266424379


In [226]:
metrics.confusion_matrix(y_test, y_pred)

array([[1205,    3],
       [  11,  174]], dtype=int64)

Although right now the precision isn't at 100%, we can see that the recall is vastly superior to the Random Forest. If I can tune this model to get the precision to be 100%, it's looking as though this will be the ideal model. Also, seeing that the cross-validated score is higher than our test score, it's safe to assume that the lower score is due to the particular train test split I have.

### SVM

In [190]:
p = Pipeline([('v', CountVectorizer(lowercase=True)), ('svc', SVC(gamma=1))])

g = GridSearchCV(p, param_grid={'v__min_df':[1,2], 'v__ngram_range':  [(1,1), (1,2)], 'v__stop_words': [None, 'english'], 
                                'v__max_features':[None, 25000]}, cv=kfold, scoring=score)
g.fit(X, y)
g.best_estimator_

Pipeline(memory=None,
     steps=[('v', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        str...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [191]:
g.best_score_

0.9795245681536897

In [210]:
v3=CountVectorizer(lowercase=True, stop_words='english')

In [211]:
X_train_f3=v3.fit_transform(X_train)
X_test_f3=v3.transform(X_test)
svc=SVC(gamma=1)
svc.fit(X_train_f, y_train)
print('F-beta score: %s' %(fbeta_score(y_test, y_pred, beta=0.1)))

F-beta score: 0.9936309354563548


In [212]:
metrics.confusion_matrix(y_test, y_pred)

array([[1207,    1],
       [  12,  173]], dtype=int64)

This model seems like the best one so far. It only filtered out one legitimate message while keeping recall much lower than the Random Forest.

### Step 3: Apply Dimensionality Reduction to Data to Streamline Model Testing

Now that I have my optimal vectorization, I will use PCA to see if I can reduce the dimensions of my features without losing too much precision.

In [52]:
# Create a pipeline of PCA and a model to test on (I chose Random Forest)
# Choose between different powers of 10 to get a broad range of components

p = Pipeline([('pca', PCA()), ('forest', RandomForestClassifier())])
g = GridSearchCV(p, param_grid={'pca__n_components':[100,1000,10000]}, cv=kfold, scoring='precision')
g.fit(X,y)
g.best_estimator_

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=1000, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
      ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

My Grid search revealed that 1000 features is optimal, so I will now apply PCA, make a new test/train split, and test it on a Random Forest to see how my metrics have changed.

In [54]:
pca=PCA(n_components=1000)
X_reduced=pca.fit_transform(X)
X_reduced

array([[-0.69031461, -0.06697618,  0.1153801 , ...,  0.05144084,
        -0.09045583, -0.0082527 ],
       [-0.86246656, -0.19513463, -0.04169248, ...,  0.01479478,
        -0.03191906, -0.02423177],
       [ 0.75495665,  2.05805881, -1.88678097, ..., -0.00460208,
        -0.03985623,  0.04602284],
       ...,
       [-0.45910366, -0.03230226,  0.270393  , ..., -0.01793328,
         0.05162029,  0.06291157],
       [ 0.68689306,  1.02529781,  0.4527041 , ..., -0.28648467,
         0.14663249,  0.0219222 ],
       [-0.33968641,  0.43082418, -0.43010495, ..., -0.04125324,
         0.0092583 , -0.00432963]])

In [55]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_reduced, y, random_state=8)
forest=RandomForestClassifier()
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
print('F-beta score: %s' %(fbeta_score(y_test, y_pred, beta=0.1)))

Accuracy: 0.9425699928212491
Precision: 0.9206349206349206 
Recall: 0.6236559139784946


Although my metrics did not go down too much, my overarching goal is to keep precision at 100%, which seems impossible with dimensionality reduction, so I am going to go ahead without it. However, I can still use the knowledge that 1000 features is the optimal number when tuning my models.

## Step 4: Test and Tune Models to Find the Best One

Keeping all of my features, I will go ahead and tune my models to find out which performs better overall. I will be cross-validating all of my models to ensure that none of them are overfitting the particular test-train split I have.

### Random Forest

Since I'm looking for perfect precision, I'm not going to mess with the size of the leaves, so I will focus my tuning on the number of trees and the maximum features in each tree. Because dimensionality reduction recommended reducing to 1000 features, I can use that information in my tuning and test a number of features between 500 and 5000 to pinpoint the ideal number. With the number of trees, I am going to test 50, 100, and 200 trees.

In [262]:
p = Pipeline([('v', v), ('forest', RandomForestClassifier())])

g = GridSearchCV(p, param_grid={'forest__n_estimators': [50,100,200],
                  'forest__max_features': [500,1000,2000,5000]}, cv=kfold, scoring=score)
g.fit(X, y)
g.best_estimator_

Pipeline(memory=None,
     steps=[('v', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_a...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [263]:
print(g.best_estimator_.named_steps)
print(g.best_score_)

{'v': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None), 'forest': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=500, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)}
0.9948708051435686


Surprisingly, the best f-beta score I got was with just 500 features. The number of trees recommended was the highest amount I tested, 200, which is unsurprising.

The best f-beta score I could get with the Random Forest is 99.47%.

### Naive Bayes

Naive Bayes is easy to tune because there is only one hyperparameter, which is alpha. I will test various negative degrees of 10.

In [227]:
p = Pipeline([('v', v2), ('nb', MultinomialNB())])

g = GridSearchCV(p, param_grid={'nb__alpha': [.001,.01,.1,1]}, cv=kfold, scoring=score)
g.fit(X, y)
g.best_estimator_

Pipeline(memory=None,
     steps=[('v', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb', MultinomialNB(alpha=1, class_prior=None, fit_prior=True))])

In [228]:
g.best_score_

0.9918029157032761

It appears that the default alpha of 1 is optimal for the best f-beta score.

The best f-beta score I could get with Naive Bayes is 99.18%.

### SVM

SVM has 2 hyperparameters, C and gamma, so I will perform a Grid search to determine the optimal combination. 

In [246]:
p = Pipeline([('v', v3), ('svc', SVC())])

g = GridSearchCV(p, param_grid={'svc__gamma': [ 0.001, 0.01, 0.1, 1],
                  'svc__C': [1, 10, 100, 1000]}, cv=kfold, scoring=score)
g.fit(X, y)
g.best_estimator_

Pipeline(memory=None,
     steps=[('v', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        str...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [247]:
print(g.best_estimator_.named_steps)
print(g.best_score_)

{'v': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None), 'svc': SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)}
0.9970005396105206


The optimal combination of C and gamma are C=10 and gamma=0.1. This provides an optimal f-beta score of 99.7%


## Conclusion:

To conclude, my SVM model was able to just barely edge out my Random Forest model for the highest f-beta score, but all three scores were over 99%, which is great. Let's take a look at the confusion matrix for each with the final tuning parameters:

In [261]:
forest=RandomForestClassifier(n_estimators=100, max_features=500)
forest.fit(X_train_f, y_train)
y_pred = forest.predict(X_test_f)
metrics.confusion_matrix(y_test,y_pred)

array([[1208,    0],
       [  25,  160]], dtype=int64)

In [223]:
nb=MultinomialNB()
nb.fit(X_train_f2, y_train)
y_pred = nb.predict(X_test_f2)
metrics.confusion_matrix(y_test, y_pred)

array([[1205,    3],
       [  11,  174]], dtype=int64)

In [249]:
svc=SVC(C=10, gamma=0.1)
svc.fit(X_train_f3, y_train)
y_pred = svc.predict(X_test_f3)
metrics.confusion_matrix(y_test, y_pred)

array([[1208,    0],
       [  21,  164]], dtype=int64)

As you can see, my SVM model was able to filter out spam without filtering out any legitimate messages, and only let 21 spam messages through the filter. My Random Forest model did nearly as well, but let 25 spam messages through the filter. While my Naive Bayes model only let 11 spam messages through the filter, it also accidentally filtered out 3 legitimate messages, making it the least desirable model of the three. The next step would be to find those three messages that the filter accidentally picked up and analyze them to see if we can gain any insights about them.